In [ ]:
import pandas as pd
import random
filepath="c/users/pkuzi/documents"
a=pd.read_csv(filepath+'Dataset.csv')
a.drop(columns = 'URL', axis = 1, inplace= True)
a.drop(columns = 'Artmis', axis = 1, inplace= True)
a.drop(columns = 'Google Safebrowsing', axis = 1, inplace= True)
a.drop(columns = 'Fraudscore', axis = 1, inplace= True)
a.drop(columns = 'CyberCrime', axis = 1, inplace= True)
a.drop(columns = 'ESET', axis = 1, inplace= True)
b=pd.read_csv(filepath+'Labels.csv')

a=a.values.tolist()
b=b['Labels'].tolist()



In [ ]:
# import packages
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
 
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

from typing import List

class Generator(nn.Module):

    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))



class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1);
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))


# Models
generator = Generator(7)
discriminator = Discriminator(7)

def train(max_int: int = 128, batch_size: int = 10, training_steps: int = 25):
    input_length = 7


    # Optimizers
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

    # loss
    loss = nn.BCELoss()
    for i in range(0,1000):
      for i in range(training_steps):
          # zero the gradients on each iteration
          generator_optimizer.zero_grad()

          # Create noisy input for generator
          # Need float type instead of int
          noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
          generated_data = generator(noise)

          # Get next data
          true_labels = b[i*10:10+i*10]
          true_data = a[i*10:10+i*10]


          true_labels = torch.tensor(true_labels).float()
          true_data = torch.tensor(true_data).float()

          # Train the generator
          generator_discriminator_out = discriminator(generated_data)

          generator_loss = loss(generator_discriminator_out, true_labels.unsqueeze(1))
          generator_loss.backward()
          generator_optimizer.step()

          # Train the discriminator on the true/generated data
          discriminator_optimizer.zero_grad()
          true_discriminator_out = discriminator(true_data)
          true_discriminator_loss = loss(true_discriminator_out, true_labels.unsqueeze(1))

          generator_discriminator_out = discriminator(generated_data.detach())
          generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size).unsqueeze(1))
          discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
          discriminator_loss.backward()
          discriminator_optimizer.step()

train()